In [9]:
import pandas as pd


def read_tables():
    url = "https://docs.google.com/spreadsheets/d/15nTfetJP_EO4uN7ATwVTPti12v_DppC4O49zbxgM1PE/edit?usp=sharing"
    url_pandas = url.replace("/edit?usp=sharing", "/export?format=xlsx")
    xls = pd.ExcelFile(url_pandas)

    dfs = {}
    for sheet_name in xls.sheet_names:
        df = xls.parse(sheet_name)
        df.columns = df.iloc[0]
        df = df[1:]
        dfs[sheet_name] = df

    print("total dfs:", len(dfs))
    return dfs


dfs = read_tables()

total dfs: 23


In [10]:
def interpolate_dataframes(dfs):
    interpolated_dfs = {}
    for sheet_name, df in dfs.items():
        # Forward fill string columns
        object_columns = df.select_dtypes(include=["object"]).columns
        df[object_columns] = df[object_columns].fillna(method="ffill")

        # Interpolate numeric columns
        numeric_columns = df.select_dtypes(include=["number"]).columns
        df[numeric_columns] = df[numeric_columns].interpolate()

        interpolated_dfs[sheet_name] = df

    return interpolated_dfs


# Apply interpolation to all dataframes
interpolated_dfs = interpolate_dataframes(dfs)

print("Interpolation complete. Number of processed dataframes:", len(interpolated_dfs))

# Example: Print the first few rows of the first dataframe to verify
first_df_name = list(interpolated_dfs.keys())[0]
print(f"\nFirst few rows of '{first_df_name}':")
print(interpolated_dfs[first_df_name].head())

Interpolation complete. Number of processed dataframes: 23

First few rows of 'Program Studi Magister dan Pasc':
0                     Program Studi Strata Fakultas / Sekolah  \
1  Magister Administrasi Pertanahan     S2               FITB   
2                    Teknik Geologi     S2               FITB   
3                    Sains Kebumian     S2               FITB   
4                  Teknik Air Tanah     S2               FITB   
5      Teknik Geodesi dan Geomatika     S2               FITB   

0 Tersedia pada Pendaftaran Gelombang 1 Tersedia pada Pendaftaran Gelombang 2  \
1                                 Tidak                                 Tidak   
2                                    Ya                                    Ya   
3                                    Ya                                    Ya   
4                                    Ya                                    Ya   
5                                    Ya                                    Ya   

0 Tersedi

/tmp/ipykernel_193057/580816609.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[object_columns] = df[object_columns].fillna(method='ffill')
/tmp/ipykernel_193057/580816609.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[object_columns] = df[object_columns].fillna(method='ffill')
/tmp/ipykernel_193057/580816609.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[object_columns] = df[object_columns].fillna(method='ffill')
/tmp/ipykernel_193057/580816609.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[object_columns] = df[object_columns].fillna(method='ffill')
/tmp/ipykernel_193057/580816609.py:6: Future

In [12]:
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.types import String, Float, Integer


def create_postgresql_tables(dfs):
    # Replace these with your actual PostgreSQL connection details
    db_params = {
        "dbname": "testdb",
        "user": "postgres",
        "password": "password",
        "host": "localhost",
        "port": "5432",
    }

    # Create a SQLAlchemy engine
    engine = create_engine(
        f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
    )

    for table_name, df in dfs.items():
        # Clean the table name (remove spaces, special characters)
        clean_table_name = "".join(e for e in table_name if e.isalnum()).lower()

        # Determine column types
        dtype_dict = {}
        for column in df.columns:
            if df[column].dtype == "object":
                dtype_dict[column] = String()
            elif df[column].dtype == "float64":
                dtype_dict[column] = Float()
            elif df[column].dtype == "int64":
                dtype_dict[column] = Integer()

        # Insert the dataframe into PostgreSQL
        df.to_sql(
            clean_table_name, engine, if_exists="replace", index=False, dtype=dtype_dict
        )
        print(f"Table '{clean_table_name}' created and populated in PostgreSQL.")


# Call the function with our interpolated dataframes
create_postgresql_tables(interpolated_dfs)

print("All tables have been created and populated in PostgreSQL.")

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  database "testdb" does not exist

(Background on this error at: https://sqlalche.me/e/20/e3q8)